# Generate Time Series CSV to be Merged in QGIS with a Point Layer.

In QGIS we created a point layer representing environmental measurement stations named `my_stations`. One of the point layer attributes is named `info`. In fact it contains the station id, i.e. "S01", "S02", "S03", etc. 

In this notebook sinusoidal time series are created as test data. The goal is to learn how to use the temporal manager in QGIS. The time series data must be provided in a 1:N relation, i.e. with the relation schema (station_id (sid), timestamp (ts), value (val)). In this notebook the relation is named `df_long` and exported as `ts_sin.csv` (time series sinusoidal). 

This csv file is to be imported in QGIS as delimited text file without coordinate into, i.e. just a table. The relations `my_stations` and `ts_sin` are then joined with the condition `ts_sin(sid) == my_stations(info)`.

The resulting new point layer in QGIS looks in principle (but not exactly) like:

| sid | name |  ts | val |  lat | lon |
|-----------:|---:|-----------:|-----------:|-----:|----:|
|         S01 | Duisburg | 1980-01-01 |       12.3 | 51.2 | 7.0 |
|         S01 | Duisburg | 1981-01-01 |       11.3 | 51.2 | 7.0 |
|         S01 | Duisburg | 1982-01-01 |       11.7 | 51.2 | 7.0 |
|        ... |        ... |        ... |  ... | ... |
|         S10 | Herne | 1980-01-01 |       13.5 | 50.7 | 6.5 |
|         S10 | Herne | 1981-01-01 |       12.7 | 50.7 | 6.5 |
|         S10 | Herne | 1982-01-01 |       13.1 | 50.7 | 6.5 |
|        ... |        ... |        ... |  ... | ... |

**Result: Many points are plotted on top of each other! There are several "Duisburg" point snow with different values.** 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dr = pd.date_range("2022-03-01T00:00:00", 
                   "2022-03-03T00:00:00",
                   freq="h",
                   tz="UTC",
                   inclusive="both")

In [3]:
dr.hour

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
            10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,  0],
           dtype='int64')

In [4]:
#help(pd.date_range)

In [5]:
#t = np.arange(0.,24.000001,0.5)
t = dr.hour
t

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
            10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,  0],
           dtype='int64')

## Exercise: Increase the temporal resolution of the date range.

Replace argument `freq="h"` with `periods=1000`. 

Calculate fractional hours (float) from `dr.hour`, `dr.minute`, `dr.second`

`th = dr.hour + ...`

In [6]:
# your code
# t = dr.hour + ...

## Generate your own time series.

**Function to create sinusoidal time series**

$$
p(t,A,A_0,T,\Delta t) = A\sin(\frac{2\pi}{T}(t-\Delta t))+A_0
$$

In [7]:
def p(t, A=1., A0=0., T=24., Dt=0.):
    return A*np.sin(2*np.pi/T*(t-Dt))+A0

**Create time series with different phases (time shift)**

In [8]:
A0=1.
p01 = p(t, A0=A0)
p02 = p(t, Dt=3., A0=A0)
p03 = p(t, Dt=6., A0=A0)
p04 = p(t, Dt=9., A0=A0)
p05 = p(t, Dt=12., A0=A0)
p06 = p(t, Dt=15., A0=A0)
p07 = p(t, Dt=18., A0=A0)
p08 = p(t, Dt=21., A0=A0)
#p09 = p(t, Dt=24., A0=A0)

NameError: name 'A0a' is not defined

In [ ]:
fig1, ax1 = plt.subplots(figsize=(12,4), dpi=136)
ax1.plot(dr,p01)
ax1.plot(dr,p02)
ax1.grid(True)
plt.show()

**Combine all series in a "wide" dataframe. Start with `dict()`.** 

In [ ]:
data_dict = {
    "S01" : p01, 
    "S02" : p02,
    "S03" : p03,
    "S04" : p04,
    "S05" : p05,
    "S06" : p06,
    "S07" : p07,
    "S08" : p08
}

In [ ]:
df = pd.DataFrame.from_dict(data_dict)

In [ ]:
df.set_index(dr, inplace=True)

In [ ]:
df.head()

In [ ]:
fig2, ax2 = plt.subplots(figsize=(12,4), dpi=136)
df.plot(ax=ax2)
plt.show()

In [ ]:
type(df["S01"])

In [ ]:
df.columns

**Select all series and print one after the other.**

In [ ]:
for sid in df.columns:
    df_tmp = pd.DataFrame(df[sid])
    df_tmp["sid"]=sid
    df_tmp.rename(columns={sid:"val"},inplace=True)
    print(df_tmp.head())

In [ ]:
#df_tmp

## Exercise: Concatenate all series in a persistent "long" dataframe.

**Save `df_long` to `./data/ts_sin.csv`.**

In [ ]:
#df_long.to_csv("./data/ts_sin.csv")